Pobieranie wiadomości z tvn24.pl
Zapis do pliku ./data/tvn24_{timestamp}.txt

In [1]:
# ! pip install bs4
# ! pip install requests
# ! pip install tqdm
# ! pip install pandas
# ! pip install pyarrow
# ! pip install qgrid
# ! pip install blaze
# ! pip install ipynbname

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, trange
from datetime import datetime, timezone
from common.webarticle import WebArticle
from common.pl_helper import PolishLocaleHelper
import pandas as pd
import logging
import ipynbname

In [3]:
# settings:
portal_name = 'tvn24'
website = 'https://tvn24.pl/polska'
pagination_attributes = '/'
no_articles_to_scrap = 300
data_dir = '../data'
logging.basicConfig(
    filename=f'../logs/{ipynbname.name()}.ipynb.log',
    encoding='utf-8',
    level=logging.DEBUG
)
ignored_paragraph_starts = ['OGLĄDAJ TELEWIZJĘ NA ŻYWO W TVN24 GO >>>']

Sprawdźmy plik robots.txt, w czasie pisania tego kodu pozwalał na pobieranie plików z portalu tvn24 bez specjalnych ograniczeń

In [4]:
resp = requests.get('https://www.tvn24.pl/robots.txt')
print(resp.text)

User-agent: *
Disallow:



In [5]:
def check_if_ignored_paragraph(text, ignored_paragraphs, operation='startswith'):

    for ignored_paragraph in ignored_paragraphs:
        if operation == 'startswith':
            if text.startswith(ignored_paragraph):
                return True
        else:
            logging.debug(f'check_if_ignored: operation {operation} not implemented')
    return False


In [6]:
def scrap_article_text_tvn24(link):
    logging.info('scrap_article_text_tvn24:')
    logging.info(f'article link to scrap: {link}')

    page = requests.get(link)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return '', ''

    soup = BeautifulSoup(page.content, 'html.parser')
    paragraphs = soup.find_all('p', class_='paragraph')

    text = ''
    for paragraph in paragraphs:
        paragraph_text = paragraph.get_text().strip()
        if check_if_ignored_paragraph(paragraph_text, ignored_paragraph_starts):
            continue
        text += paragraph_text + ' '

    time_string = soup.find('time', class_='article-top-bar__date').get_text().strip()
    when_published = PolishLocaleHelper.parse_pl_date_tvn24(time_string)

    title_int = soup.find('h1', class_='article-top-bar__title').get_text().strip()

    lead_text_int_candidate = soup.find('p', class_='lead-text')
    if lead_text_int_candidate is not None:
        lead_text_int = lead_text_int_candidate.get_text().strip()
    else:
        lead_text_int = soup.find_all('div', class_='article-element article-element--subhead')[0].get_text().strip()

    author = soup.find_all('dd', class_='article-top-bar__meta-item')[0].get_text().strip()
    source = soup.find_all('dd', class_='article-top-bar__meta-item')[1].get_text().strip()

    return text, when_published, title_int, lead_text_int, author, source

In [7]:
def scrap_articles_tvn24(website):
    logging.info('scrap_articles_tvn24:')
    logging.info(f'page with articles to scrap: {website}')

    result = list()
    page = requests.get(website)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return result

    soup = BeautifulSoup(page.content, 'html.parser')
    page_content = soup.find_all('article',class_='teaser-wrapper__content default-teaser default-teaser--standard')

    print(f'articles to scrap: {len(page_content)}')

    for article in page_content:

        title = article.find('h2',class_='heading article-title').get_text().strip()
        lead_text = article.find('p',class_='article-lead__text').get_text().strip()
        link = article.find_all('a', href=True)[0]['href']
        webart = WebArticle(title, lead_text, link, *scrap_article_text_tvn24(link),portal_name)
        logging.info(f'webarticle scrapped: {webart}')
        result.append(webart)

    return result

In [8]:
no_scrapped_articles = 0
page_no=0
scrapped_articles = list()

logging.info(f'no_articles_to_scrap: {no_articles_to_scrap}')

pbar = tqdm(total=no_articles_to_scrap)
while no_scrapped_articles < no_articles_to_scrap:
    logging.info('next page: '+website+pagination_attributes+str(page_no))
    new_articles = scrap_articles_tvn24(website+pagination_attributes+str(page_no))
    page_size = len(new_articles)
    logging.info(f'new_articles: {page_size}')
    if no_scrapped_articles + page_size > no_articles_to_scrap:
        page_size = no_articles_to_scrap - no_scrapped_articles
    scrapped_articles.extend(new_articles[:page_size])
    logging.info(f'scrapped new articles: {page_size}')
    no_scrapped_articles += page_size
    pbar.update(page_size)
    logging.info(f'no_scrapped_articles (total): {no_scrapped_articles}')
    page_no += 1
    if len(scrapped_articles) <= 0:
        logging.debug(f'0 articles scrapped!')
        break
pbar.close()

  0%|          | 0/300 [00:00<?, ?it/s]

articles to scrap: 19


  6%|▋         | 19/300 [00:16<04:08,  1.13it/s]

articles to scrap: 19


 13%|█▎        | 38/300 [00:30<03:24,  1.28it/s]

...

 94%|█████████▎| 281/300 [05:12<00:27,  1.47s/it]

articles to scrap: 17


 99%|█████████▉| 298/300 [05:41<00:03,  1.54s/it]

articles to scrap: 17


100%|██████████| 300/300 [06:08<00:00,  1.23s/it]


In [9]:
for i in trange(len(scrapped_articles)):
    # print(f'{i}: {scrapped_articles[i].lead_text}')
    print(f'{i}: {len(scrapped_articles[i].text)}')

100%|██████████| 300/300 [00:00<00:00, 13044.29it/s]

0: 2075
1: 4006
2: 3326
3: 1835
4: 3787
5: 1266
6: 5770
7: 6903
8: 3041
9: 2020
10: 554
11: 973
12: 3456
13: 3524
14: 4408
15: 2824
16: 990
17: 2707
18: 2481
19: 2075
20: 4006
21: 3326
22: 1835
23: 3787
24: 1266
25: 5770
26: 6903
27: 3041
28: 2020
29: 554
30: 973
31: 3456
32: 3524
33: 4408
34: 2824
35: 990
36: 2707
37: 2481
38: 2267
39: 1672
40: 901
41: 5142
42: 4765
43: 2795
44: 630
45: 2496
46: 1502
47: 4755
48: 927
49: 1808
50: 2653
51: 8101
52: 3259
53: 2737
54: 8483
55: 729
56: 3567
57: 379
58: 4939
59: 1946
60: 1196
61: 2684
62: 1597
63: 3000
64: 1248
65: 1133
66: 6478
67: 707
68: 10557
69: 2753
70: 2370
71: 3602
72: 1401
73: 1245
74: 1736
75: 2134
76: 822
77: 1100
78: 4305
79: 8578
80: 3196
81: 1612
82: 3770
83: 342
84: 2307
85: 783
86: 2195
87: 709
88: 1273
89: 1035
90: 7135
91: 2736
92: 3346
93: 1243
94: 1618
95: 3286
96: 1626
97: 977
98: 3935
99: 2717
100: 960
101: 1643
102: 442
103: 3829
104: 4275
105: 657
106: 4656
107: 9116
108: 1221
109: 605
110: 967
111: 5686
112: 2812
1

In [10]:
tvn24_dataframe = pd.DataFrame(article.__dict__ for article in scrapped_articles)
tvn24_dataframe

C:\Users\piotr\AppData\Local\Temp/ipykernel_16860/2017229850.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  tvn24_dataframe = pd.DataFrame(article.__dict__ for article in scrapped_articles)


,title,title_int,lead_text,lead_text_int,link,text,when_published,author,source,portal
0,Czwartek pod znakiem silnego wiatru i opadów ś...,Pogoda na dziś - czwartek 20.01. Dzień pod zna...,Pogoda na dziś.,Pogoda na dziś zapowiada się pochmurno. Należy...,https://tvn24.pl/tvnmeteo/prognoza/pogoda-na-d...,Pogoda na dziś przyniesie nam duże zachmurzeni...,2022-01-20 02:00:00,kw,tvnmeteo.pl,tvn24
1,"Silny wiatr, oblodzenie, zawieje lub zamiecie ...","Alerty IMGW - silny wiatr, oblodzenie, zawieje...","Sprawdź, gdzie wydano alarmy.",IMGW ostrzega - w wielu regionach kraju pogoda...,https://tvn24.pl/tvnmeteo/prognoza/alerty-imgw...,Synoptycy Instytutu Meteorologii i Gospodarki ...,2022-01-19 06:23:00,"kw,ps/dd",IMGW,tvn24
2,"""Wartości zakażeń, które są w tej chwili podaw...",Ekspertka od modeli matematycznych: podawane w...,Doktor Aneta Afelt i doktor nauk medycznych Ma...,"O epidemii COVID-19, liczbie zakażeń koronawir...",https://tvn24.pl/polska/koronawirus-w-polsce-d...,Do środy w Polsce potwierdzono 4 373 718 zakaż...,2022-01-19 21:35:00,akr,TVN24,tvn24
3,WOŚP 2022. Na placu Defilad stanął wielki nami...,WOŚP 2022. Na placu Defilad stanął wielki nami...,Będą to znani polscy artyści.,Jubileuszowy 30. Finał Wielkiej Orkiestry Świą...,https://tvn24.pl/tvnwarszawa/najnowsze/wosp-20...,30. Finał Wielkiej Orkiestry Świątecznej Pomoc...,2022-01-19 21:04:00,dg/gp,tvnwarszawa.pl,tvn24
4,Sikorski o sejmowej komisji: nie mając nic do ...,Sejmowa komisja w sprawie Pegasusa. Radosław S...,"Europoseł był gościem ""Kropki nad i"".","Radosław Sikorski, europoseł Platformy Obywate...",https://tvn24.pl/polska/pegasus-sejmowa-komisj...,"""KROPKA NAD I"" - OGLĄDAJ W TVN24 GO >>> Grupa ...",2022-01-19 20:50:00,mjz,TVN24,tvn24
...,...,...,...,...,...,...,...,...,...,...
295,Możliwe osiągnięcie stanów ostrzegawczych. Ost...,Możliwe osiągnięcie stanów ostrzegawczych. Ost...,Instytut Meteorologii i Gospodarki Wodnej wyda...,Poziom wód w rejonie Żuław i Zalewu Wiślanego ...,https://tvn24.pl/tvnmeteo/prognoza/imgw-alerty...,"""W związku z prognozowanym silnym, sztormowym ...",2022-01-15 06:55:00,ps,"PAP, IMGW",tvn24
296,NFZ obcina stawki za testy na koronawirusa. Cz...,NFZ obcina stawki za testy na obecność koronaw...,"Materiał magazynu ""Polska i Świat"".",Narodowy Fundusz Zdrowia obciął stawki na poje...,https://tvn24.pl/polska/koronawirus-w-polsce-n...,"Najpierw wymaz, a później badanie w budynku tu...",2022-01-15 06:32:00,AleksandraKąkol,TVN24,tvn24
297,Pogoda na dziś. Termometry pokażą od 2 do 5 st...,Pogoda na dziś - sobota 15.01. Zachmurzenie bę...,W sobotę w całym kraju odczujemy chłód.,Pogoda na dziś. Sobota (15.01) upłynie pod zna...,https://tvn24.pl/tvnmeteo/prognoza/pogoda-na-d...,Na sobotę prognozowane jest zmienne zachmurzen...,2022-01-15 02:00:00,anw,tvnmeteo.pl,tvn24
298,"""Polskie prawo w sprawie inwigilacji nie odpow...",Rzecznik Praw Obywatelskich interweniuje u pre...,RPO interweniuje u premiera.,Rzecznik Praw Obywatelskich Marcin Wiącek wyst...,https://tvn24.pl/polska/pegasus-rzecznik-praw-...,OGLĄDAJ TVN24 W INTERNECIE W TVN24 GO Marcin W...,2022-01-14 22:49:00,mjz//rzw,PAP,tvn24


In [11]:
timestamp = int((datetime.now(timezone.utc)).timestamp())

In [37]:
tvn24_dataframe.to_parquet(f'{data_dir}/{portal_name}_{timestamp}.parquet')
tvn24_dataframe.to_json(f'{data_dir}/{portal_name}_{timestamp}.json')

ArrowTypeError: ("Expected bytes, got a 'WebArticle' object", 'Conversion failed for column title with type object')

Krótki test - dane pierwszego artykułu

In [13]:
print(f'title:\n  {tvn24_dataframe.iloc[0]["title"]}')
print(f'lead_text:\n  {tvn24_dataframe.iloc[0]["lead_text"]}')
print(f'link:\n  {tvn24_dataframe.iloc[0]["link"]}')
print(f'text:\n  {tvn24_dataframe.iloc[0]["text"]}')
print(f'when_published:\n  {tvn24_dataframe.iloc[0]["when_published"]}')

title:
  Czwartek pod znakiem silnego wiatru i opadów śniegu
lead_text:
  Pogoda na dziś.
link:
  https://tvn24.pl/tvnmeteo/prognoza/pogoda-na-dzis-czwartek-2001-dzien-pod-znakiem-silnego-wiatru-i-opadow-sniegu-5565971
text:
  Pogoda na dziś przyniesie nam duże zachmurzenie z większymi przejaśnieniami. Wystąpią przelotne opady śniegu do 1-5 centymetrów, w górach - do 10 cm. Początkowo, szczególnie na południu, pojawią się również opady deszczu ze śniegiem. Na termometrach zobaczymy maksymalnie od -1 stopnia Celsjusza na północnym wschodzie kraju do 2 st. C na Podkarpaciu. Zachodni i północno-zachodni wiatr będzie silny - w porywach może rozpędzać się do 50-70 kilometrów na godzinę. W niektórych miejscach na południu porywy mogą sięgać jednak do 90 km/h, a na Wybrzeżu - do 100 km/h. Wilgotność powietrza w ciągu dnia znajdzie się na poziomie 60-80 procent, przy czym najwyższa będzie na południu, w centrum kraju oraz na wschodzie. Mieszkańcy całej Polski odczują zimno, a uczucie chłodu bę

In [34]:
print(f'title:\n  {tvn24_dataframe.iloc[172]["title"]}')
print(f'lead_text:\n  {tvn24_dataframe.iloc[172]["lead_text"]}')
print(f'link:\n  {tvn24_dataframe.iloc[172]["link"]}')
print(f'text:\n  {tvn24_dataframe.iloc[172]["text"]}')
print(f'when_published:\n  {tvn24_dataframe.iloc[172]["when_published"]}')
print(f'portal:\n  {tvn24_dataframe.iloc[172]["portal"]}')

title:
  Śmiertelny wypadek na S1. Zderzenie ciągnika, osobówki i busa
lead_text:
  Cztery osoby w szpitalu, jedna nie żyje.
link:
  https://tvn24.pl/katowice/slaskie-zywiec-trasa-s1-smiertelny-wypadek-z-udzialem-ciagnika-osobowki-i-busa-5563116
text:
  Do tragicznego wypadku na trasie S1 doszło około godz. 20. Według informacji dyżurnego Wojewódzkiego Stanowiska Koordynacji Ratownictwa, kierujący ciągnikiem z naczepą na rozwidleniu drogi nie zastosował się do oznakowania. Wjechał na trasę pod prąd i tam zderzył się czołowo z samochodem osobowym, a następnie uderzył w busa. OGLĄDAJ NA ŻYWO W TVN24 GO Cztery osoby - pasażerka samochodu osobowego, kierowca samochodu ciężarowego i dwie osoby podróżujące busem - zostały zabrane do szpitala. Niestety zderzenia nie przeżył kierujący samochodem osobowym. Dyżurny WSKR poinformował, że na miejscu wypadku przybył prokurator i cztery zastępy straży pożarnej. Strażacy, poza zabezpieczeniem miejsca wypadku, musieli między innymi wypompowywać olej n

In [40]:
# fix article 172 - did not load properly
link172 = 'https://tvn24.pl/katowice/slaskie-zywiec-trasa-s1-smiertelny-wypadek-z-udzialem-ciagnika-osobowki-i-busa-5563116'
title = 'Śmiertelny wypadek na S1. Zderzenie ciągnika, osobówki i busa'
result = scrap_article_text_tvn24(link172)
tvn24_dataframe.at[172,'title'] = title
tvn24_dataframe.at[172,'lead_text'] = 'Cztery osoby w szpitalu, jedna nie żyje.'
tvn24_dataframe.at[172,'link'] = link172
tvn24_dataframe.at[172,'text'] = result[0]
tvn24_dataframe.at[172,'when_published'] = result[1]
tvn24_dataframe.at[172,'title_int'] = result[2]
tvn24_dataframe.at[172,'lead_text_int'] = result[3]
tvn24_dataframe.at[172,'author'] = result[4]
tvn24_dataframe.at[172,'source'] = result[5]
tvn24_dataframe.at[172,'portal'] = portal_name

tvn24_dataframe.to_parquet(f'{data_dir}/{portal_name}_{timestamp}.parquet')
tvn24_dataframe.to_json(f'{data_dir}/{portal_name}_{timestamp}.json')

In [39]:
result = scrap_article_text_tvn24(link172)
print(f'text: {result[0]}')
print(f'when_published: {result[1]}')
print(f'title_int: {result[2]}')
print(f'lead_text_int: {result[3]}')
print(f'author: {result[4]}')
print(f'source: {result[5]}')
print(type(result[1]))
print(type(tvn24_dataframe.iloc[172]["when_published"]))

text: Do tragicznego wypadku na trasie S1 doszło około godz. 20. Według informacji dyżurnego Wojewódzkiego Stanowiska Koordynacji Ratownictwa, kierujący ciągnikiem z naczepą na rozwidleniu drogi nie zastosował się do oznakowania. Wjechał na trasę pod prąd i tam zderzył się czołowo z samochodem osobowym, a następnie uderzył w busa. OGLĄDAJ NA ŻYWO W TVN24 GO Cztery osoby - pasażerka samochodu osobowego, kierowca samochodu ciężarowego i dwie osoby podróżujące busem - zostały zabrane do szpitala. Niestety zderzenia nie przeżył kierujący samochodem osobowym. Dyżurny WSKR poinformował, że na miejscu wypadku przybył prokurator i cztery zastępy straży pożarnej. Strażacy, poza zabezpieczeniem miejsca wypadku, musieli między innymi wypompowywać olej napędowy z ciężarówki. Późnym wieczorem droga wciąż była zablokowana. 
when_published: 2022-01-17 23:17:00
title_int: Śmiertelny wypadek na trasie S1. Zderzenie ciągnika, osobówki i busa
lead_text_int: Na trasie S1 w Żywcu, w kierunku Węgierskiej Gó

In [35]:
tvn24_dataframe

,title,title_int,lead_text,lead_text_int,link,text,when_published,author,source,portal
0,Czwartek pod znakiem silnego wiatru i opadów ś...,Pogoda na dziś - czwartek 20.01. Dzień pod zna...,Pogoda na dziś.,Pogoda na dziś zapowiada się pochmurno. Należy...,https://tvn24.pl/tvnmeteo/prognoza/pogoda-na-d...,Pogoda na dziś przyniesie nam duże zachmurzeni...,2022-01-20 02:00:00,kw,tvnmeteo.pl,tvn24
1,"Silny wiatr, oblodzenie, zawieje lub zamiecie ...","Alerty IMGW - silny wiatr, oblodzenie, zawieje...","Sprawdź, gdzie wydano alarmy.",IMGW ostrzega - w wielu regionach kraju pogoda...,https://tvn24.pl/tvnmeteo/prognoza/alerty-imgw...,Synoptycy Instytutu Meteorologii i Gospodarki ...,2022-01-19 06:23:00,"kw,ps/dd",IMGW,tvn24
2,"""Wartości zakażeń, które są w tej chwili podaw...",Ekspertka od modeli matematycznych: podawane w...,Doktor Aneta Afelt i doktor nauk medycznych Ma...,"O epidemii COVID-19, liczbie zakażeń koronawir...",https://tvn24.pl/polska/koronawirus-w-polsce-d...,Do środy w Polsce potwierdzono 4 373 718 zakaż...,2022-01-19 21:35:00,akr,TVN24,tvn24
3,WOŚP 2022. Na placu Defilad stanął wielki nami...,WOŚP 2022. Na placu Defilad stanął wielki nami...,Będą to znani polscy artyści.,Jubileuszowy 30. Finał Wielkiej Orkiestry Świą...,https://tvn24.pl/tvnwarszawa/najnowsze/wosp-20...,30. Finał Wielkiej Orkiestry Świątecznej Pomoc...,2022-01-19 21:04:00,dg/gp,tvnwarszawa.pl,tvn24
4,Sikorski o sejmowej komisji: nie mając nic do ...,Sejmowa komisja w sprawie Pegasusa. Radosław S...,"Europoseł był gościem ""Kropki nad i"".","Radosław Sikorski, europoseł Platformy Obywate...",https://tvn24.pl/polska/pegasus-sejmowa-komisj...,"""KROPKA NAD I"" - OGLĄDAJ W TVN24 GO >>> Grupa ...",2022-01-19 20:50:00,mjz,TVN24,tvn24
...,...,...,...,...,...,...,...,...,...,...
295,Możliwe osiągnięcie stanów ostrzegawczych. Ost...,Możliwe osiągnięcie stanów ostrzegawczych. Ost...,Instytut Meteorologii i Gospodarki Wodnej wyda...,Poziom wód w rejonie Żuław i Zalewu Wiślanego ...,https://tvn24.pl/tvnmeteo/prognoza/imgw-alerty...,"""W związku z prognozowanym silnym, sztormowym ...",2022-01-15 06:55:00,ps,"PAP, IMGW",tvn24
296,NFZ obcina stawki za testy na koronawirusa. Cz...,NFZ obcina stawki za testy na obecność koronaw...,"Materiał magazynu ""Polska i Świat"".",Narodowy Fundusz Zdrowia obciął stawki na poje...,https://tvn24.pl/polska/koronawirus-w-polsce-n...,"Najpierw wymaz, a później badanie w budynku tu...",2022-01-15 06:32:00,AleksandraKąkol,TVN24,tvn24
297,Pogoda na dziś. Termometry pokażą od 2 do 5 st...,Pogoda na dziś - sobota 15.01. Zachmurzenie bę...,W sobotę w całym kraju odczujemy chłód.,Pogoda na dziś. Sobota (15.01) upłynie pod zna...,https://tvn24.pl/tvnmeteo/prognoza/pogoda-na-d...,Na sobotę prognozowane jest zmienne zachmurzen...,2022-01-15 02:00:00,anw,tvnmeteo.pl,tvn24
298,"""Polskie prawo w sprawie inwigilacji nie odpow...",Rzecznik Praw Obywatelskich interweniuje u pre...,RPO interweniuje u premiera.,Rzecznik Praw Obywatelskich Marcin Wiącek wyst...,https://tvn24.pl/polska/pegasus-rzecznik-praw-...,OGLĄDAJ TVN24 W INTERNECIE W TVN24 GO Marcin W...,2022-01-14 22:49:00,mjz//rzw,PAP,tvn24
